In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [10]:
# Step 1: Load the Data
vehicles_df = pd.read_csv('data/dataset/travels.csv')
reviews_df = pd.read_csv('data/dataset/reviews.csv')

In [11]:
vehicles_df.head()

,vehicle_id,travel_id,departure_at,from,to
0,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ace8,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu"
1,65df28d7d5f3ecff2331ad58,65df28d8d5f3ecff2331af8d,2024-03-04,"Kerkha, Jhapa","Biratnagar, Sunsari"
2,65df28d7d5f3ecff2331ad6f,65df28d8d5f3ecff2331afb3,2024-03-04,"Pathri, Morong","Pokhara, Kaski"
3,65df28d7d5f3ecff2331ada5,65df28d8d5f3ecff2331b015,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu"
4,65df28d7d5f3ecff2331adb8,65df28d8d5f3ecff2331b021,2024-03-04,"Koteshowr, Kathmandu","Pathri, Morong"


In [12]:
reviews_df.rename(columns={"id":'review_id'},inplace=True)

In [13]:
reviews_df.head()

,review_id,vehicle_id,user_id,rating
0,65df29333b3f37f4faa838a0,65df28d8d5f3ecff2331b041,11,5
1,65df29333b3f37f4faa838a1,65df28d8d5f3ecff2331b6c3,27,5
2,65df29333b3f37f4faa838a2,65df28d8d5f3ecff2331b6e8,5,5
3,65df29333b3f37f4faa838a3,65df28d7d5f3ecff2331adb8,40,5
4,65df29333b3f37f4faa838a4,65df28d8d5f3ecff2331afb1,46,3


In [14]:
print(reviews_df.shape)
print(vehicles_df.shape)

(120000, 4)
(110, 5)


In [31]:
# Step 2: Preprocess the Data
vehicles_with_reviews = pd.merge(vehicles_df, reviews_df, on='vehicle_id')

In [32]:
vehicles_with_reviews

,vehicle_id,travel_id,departure_at,from,to,review_id,user_id,rating
0,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ace8,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu",65df29333b3f37f4faa83954,6,2
1,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ace8,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu",65df29333b3f37f4faa83962,1,4
2,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ace8,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu",65df29333b3f37f4faa839e4,14,4
3,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ace8,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu",65df29333b3f37f4faa83a80,12,2
4,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ace8,2024-03-04,"Biratnagar, Sunsari","Koteshowr, Kathmandu",65df29333b3f37f4faa83b12,21,5
...,...,...,...,...,...,...,...,...
119995,65df28e3d5f3ecff2331b963,65df28e3d5f3ecff2331b974,2024-03-04,"Pokhara, Kaski","Kerkha, Jhapa",65df29353b3f37f4faaa0c42,11,2
119996,65df28e3d5f3ecff2331b963,65df28e3d5f3ecff2331b974,2024-03-04,"Pokhara, Kaski","Kerkha, Jhapa",65df29353b3f37f4faaa0caf,17,2
119997,65df28e3d5f3ecff2331b963,65df28e3d5f3ecff2331b974,2024-03-04,"Pokhara, Kaski","Kerkha, Jhapa",65df29353b3f37f4faaa0ce3,6,5
119998,65df28e3d5f3ecff2331b963,65df28e3d5f3ecff2331b974,2024-03-04,"Pokhara, Kaski","Kerkha, Jhapa",65df29353b3f37f4faaa0cec,23,4


# Collaborative Filtering Recommendation System

In [33]:
# Step 4: Generate Pivot table matrix
user_ratings = vehicles_with_reviews.pivot_table(index='user_id', columns='vehicle_id', values='rating').fillna(0)

In [34]:
user_ratings

vehicle_id,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ad41,65df28d7d5f3ecff2331ad58,65df28d7d5f3ecff2331ad6f,65df28d7d5f3ecff2331ad8c,65df28d7d5f3ecff2331ada5,65df28d7d5f3ecff2331adae,65df28d7d5f3ecff2331adb2,65df28d7d5f3ecff2331adb8,65df28d7d5f3ecff2331adc0,...,65df28d8d5f3ecff2331b744,65df28d8d5f3ecff2331b770,65df28d8d5f3ecff2331b790,65df28d8d5f3ecff2331b7b1,65df28e3d5f3ecff2331b909,65df28e3d5f3ecff2331b91b,65df28e3d5f3ecff2331b92d,65df28e3d5f3ecff2331b93d,65df28e3d5f3ecff2331b951,65df28e3d5f3ecff2331b963
user_id,,,,,,,,,,,,,,,,,,,,,
1,4.000000,3.280000,3.409091,3.592593,3.411765,3.000000,3.807692,3.647059,3.250000,3.689655,...,3.428571,3.125000,3.772727,3.400000,3.055556,3.833333,3.695652,3.111111,3.520000,3.458333
2,3.227273,3.478261,3.137931,3.687500,3.368421,3.300000,3.700000,3.653846,3.529412,3.740741,...,3.541667,3.736842,3.961538,3.379310,3.882353,3.650000,3.448276,3.291667,3.333333,3.315789
3,3.518519,3.666667,3.880000,3.454545,3.482759,3.454545,3.344828,3.416667,3.258065,3.500000,...,3.789474,3.464286,3.400000,3.040000,3.590909,3.459459,3.750000,3.814815,3.291667,3.633333
4,3.571429,3.523810,3.350000,3.583333,3.708333,3.611111,3.636364,4.050000,3.600000,3.151515,...,3.677419,3.409091,3.611111,3.444444,3.250000,3.454545,3.518519,3.565217,3.840000,3.000000
5,3.588235,3.647059,3.333333,3.233333,3.555556,3.590909,3.500000,2.928571,3.631579,3.794118,...,3.321429,3.413793,3.592593,3.250000,3.518519,3.571429,3.360000,3.461538,3.400000,3.380952
6,3.545455,3.952381,3.176471,3.476190,3.677419,3.080000,3.576923,3.357143,3.857143,4.217391,...,3.772727,3.625000,3.250000,3.250000,3.400000,4.100000,3.714286,3.562500,3.619048,3.135135
7,3.666667,3.631579,3.655172,3.625000,3.642857,3.692308,3.653846,3.476190,4.000000,3.428571,...,3.636364,3.787879,3.192308,3.500000,3.074074,3.222222,3.470588,3.478261,3.409091,3.318182
8,3.823529,3.900000,3.750000,3.650000,3.454545,3.476190,3.551724,3.450000,3.428571,3.952381,...,3.571429,3.222222,3.640000,3.266667,3.740741,3.478261,3.750000,3.548387,3.333333,3.434783
9,3.913043,2.923077,3.692308,3.636364,3.411765,3.187500,3.545455,3.038462,3.761905,3.185185,...,3.473684,3.739130,3.173913,4.043478,3.714286,3.368421,3.550000,3.642857,3.833333,3.571429


In [35]:
# Explanation: https://youtu.be/3ecNC-So0r4?t=785
# Let's say that auth user have not given rating to some vehicle and it doesn't mean that user have gave 0 start rating to it
# So we will create method which will standarized the ratings given by all the users
def standardize(row):
    # takes single rows from dataframe as argument
    # now create a new row from (original rating minus(-) mean of all the ratings & divide it by the range of ratings that the user gives)
    new_row = (row - row.mean()) / (row.max() - row.min()) 
    return new_row

ratings_std = user_ratings.apply(standardize)
ratings_std

vehicle_id,65df28d6d5f3ecff2331abe6,65df28d7d5f3ecff2331ad41,65df28d7d5f3ecff2331ad58,65df28d7d5f3ecff2331ad6f,65df28d7d5f3ecff2331ad8c,65df28d7d5f3ecff2331ada5,65df28d7d5f3ecff2331adae,65df28d7d5f3ecff2331adb2,65df28d7d5f3ecff2331adb8,65df28d7d5f3ecff2331adc0,...,65df28d8d5f3ecff2331b744,65df28d8d5f3ecff2331b770,65df28d8d5f3ecff2331b790,65df28d8d5f3ecff2331b7b1,65df28e3d5f3ecff2331b909,65df28e3d5f3ecff2331b91b,65df28e3d5f3ecff2331b92d,65df28e3d5f3ecff2331b93d,65df28e3d5f3ecff2331b951,65df28e3d5f3ecff2331b963
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.399337,-0.162795,-0.142261,0.102904,-0.059158,-0.408608,0.311010,0.125634,-0.168049,0.089228,...,-0.011276,-0.420832,0.231372,-0.120227,-0.368310,0.268230,0.139366,-0.304209,0.021736,-0.000839
2,-0.322517,0.016427,-0.438260,0.230371,-0.093455,-0.153289,0.212683,0.131686,0.012494,0.133462,...,0.101819,0.227001,0.396207,-0.140845,0.367227,0.101563,-0.140012,-0.124604,-0.158618,-0.152102
3,-0.050446,0.186741,0.371784,-0.082502,-0.002981,-0.021761,-0.111605,-0.079812,-0.162838,-0.074991,...,0.349627,-0.061588,-0.094025,-0.478980,0.107952,-0.071655,0.200744,0.395791,-0.198876,0.184866
4,-0.001019,0.057602,-0.206765,0.090468,0.175512,0.111486,0.154580,0.484944,0.058105,-0.376738,...,0.237572,-0.120030,0.090278,-0.075937,-0.195328,-0.076123,-0.060683,0.147507,0.330915,-0.487208
5,0.014681,0.169016,-0.224959,-0.379605,0.054622,0.094293,0.030074,-0.515056,0.078510,0.179680,...,-0.118419,-0.115051,0.074112,-0.269708,0.043552,0.030135,-0.239708,0.044374,-0.094206,-0.082953
6,-0.025283,0.445019,-0.396190,-0.053432,0.151051,-0.340523,0.100308,-0.132890,0.224259,0.546186,...,0.332880,0.108580,-0.224977,-0.269708,-0.061885,0.510654,0.160410,0.144804,0.117434,-0.343807
7,0.087949,0.155023,0.126362,0.146429,0.123702,0.180590,0.170542,-0.026733,0.316567,-0.136840,...,0.196516,0.281040,-0.275344,-0.020574,-0.351835,-0.287326,-0.114813,0.061008,-0.085423,-0.149563
8,0.234485,0.397668,0.229876,0.180006,-0.025306,-0.003340,0.077300,-0.050088,-0.052664,0.316718,...,0.131581,-0.317891,0.115499,-0.253099,0.241246,-0.054563,0.200744,0.130765,-0.158618,-0.025830
9,0.318105,-0.485444,0.166899,0.161691,-0.059158,-0.249034,0.071576,-0.417065,0.162720,-0.347583,...,0.033837,0.229424,-0.291402,0.521020,0.217711,-0.154418,-0.025128,0.224738,0.324473,0.119175


In [36]:
# Step 4: Compute User to User Similarities
# Cosine similaritis over here will calculate over row wise means User to User 
similarity_matrix = cosine_similarity(ratings_std) 

# or if you wanted you can do Vehicles to Vehicles similarites for that you have to transpose
vehicles_similarity_matrix = cosine_similarity(ratings_std.T)

In [13]:
similarity_matrix.shape

(200, 200)

In [14]:
similarity_matrix

array([[ 1.        ,  0.16398542, -0.06160727, ...,  0.01284502,
        -0.02617014,  0.0068169 ],
       [ 0.16398542,  1.        , -0.05858037, ...,  0.06718185,
        -0.02627877,  0.13205168],
       [-0.06160727, -0.05858037,  1.        , ...,  0.03877683,
        -0.11480534, -0.04122942],
       ...,
       [ 0.01284502,  0.06718185,  0.03877683, ...,  1.        ,
        -0.18177214,  0.04045485],
       [-0.02617014, -0.02627877, -0.11480534, ..., -0.18177214,
         1.        , -0.00164825],
       [ 0.0068169 ,  0.13205168, -0.04122942, ...,  0.04045485,
        -0.00164825,  1.        ]])

In [15]:
vehicles_similarity_matrix.shape

(103, 103)

In [16]:
vehicles_similarity_matrix

array([[ 1.        ,  0.19835287,  0.10797364, ..., -0.04406638,
         0.12597894,  0.02520853],
       [ 0.19835287,  1.        , -0.02398161, ..., -0.01755617,
         0.02922688, -0.01631652],
       [ 0.10797364, -0.02398161,  1.        , ...,  0.1303837 ,
         0.06423217, -0.00981477],
       ...,
       [-0.04406638, -0.01755617,  0.1303837 , ...,  1.        ,
         0.07001278,  0.0247252 ],
       [ 0.12597894,  0.02922688,  0.06423217, ...,  0.07001278,
         1.        ,  0.07561356],
       [ 0.02520853, -0.01631652, -0.00981477, ...,  0.0247252 ,
         0.07561356,  1.        ]])

In [17]:
# Creating dataframe out of numpy array from vehicles_similarity_matrix so that we can use mor easily
vehicles_similarity_matrix_df = pd.DataFrame(vehicles_similarity_matrix, index=ratings_std.columns, columns=ratings_std.columns)
vehicles_similarity_matrix_df
# Here '65dda7765e43b9d7d3e4f4bf' similar to '65dda7765e43b9d7d3e4f4bf' with 100%
# '65dda7765e43b9d7d3e4f4bf' similar to '65dda7775e43b9d7d3e4f4d3' with 97%
# Explanation: https://youtu.be/3ecNC-So0r4?t=1072

vehicle_id,65dda7755e43b9d7d3e4f3f9,65dda7765e43b9d7d3e4f4bf,65dda7775e43b9d7d3e4f4d3,65dda7775e43b9d7d3e4f4e6,65dda7775e43b9d7d3e4f4ea,65dda7775e43b9d7d3e4f4ef,65dda7775e43b9d7d3e4f4fb,65dda7775e43b9d7d3e4f513,65dda7775e43b9d7d3e4f529,65dda7775e43b9d7d3e4f534,...,65dda7845e43b9d7d3e4fe17,65dda7845e43b9d7d3e4fe34,65dda7855e43b9d7d3e4fe52,65dda7855e43b9d7d3e4fe6f,65dda7855e43b9d7d3e4fe8c,65dda7855e43b9d7d3e4fead,65dda7855e43b9d7d3e4fecb,65dda7855e43b9d7d3e4fee8,65dda7865e43b9d7d3e4ff06,65dda7865e43b9d7d3e4ff24
vehicle_id,,,,,,,,,,,,,,,,,,,,,
65dda7755e43b9d7d3e4f3f9,1.000000,0.198353,0.107974,0.050121,-0.129653,0.026379,-0.005630,0.019951,-0.062673,-0.015780,...,-0.011232,0.011049,-0.067388,0.017294,0.136889,0.147143,-0.035366,-0.044066,0.125979,0.025209
65dda7765e43b9d7d3e4f4bf,0.198353,1.000000,-0.023982,-0.030713,-0.043227,0.009154,0.023939,0.010267,0.011186,0.099951,...,-0.004082,0.052049,0.077645,0.045717,0.006917,0.029482,0.029404,-0.017556,0.029227,-0.016317
65dda7775e43b9d7d3e4f4d3,0.107974,-0.023982,1.000000,0.014555,-0.000135,-0.076343,0.065037,-0.060054,-0.007134,0.013093,...,-0.017202,0.004370,0.029571,0.050792,0.062098,-0.149178,0.087125,0.130384,0.064232,-0.009815
65dda7775e43b9d7d3e4f4e6,0.050121,-0.030713,0.014555,1.000000,-0.113590,-0.067225,0.071979,0.020889,-0.108720,0.004932,...,-0.006995,0.031617,-0.059785,0.102149,0.110645,-0.014905,-0.014105,0.128684,0.180413,0.169692
65dda7775e43b9d7d3e4f4ea,-0.129653,-0.043227,-0.000135,-0.113590,1.000000,0.077221,0.031586,-0.004886,-0.094156,-0.071225,...,-0.104831,-0.123031,0.135604,-0.039622,-0.022971,0.028613,-0.033465,0.106229,-0.098971,-0.004707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65dda7855e43b9d7d3e4fead,0.147143,0.029482,-0.149178,-0.014905,0.028613,-0.002831,-0.086876,-0.010508,0.033319,0.042402,...,-0.084111,-0.002555,0.033203,-0.121380,0.068136,1.000000,-0.025963,-0.035056,0.004803,-0.139972
65dda7855e43b9d7d3e4fecb,-0.035366,0.029404,0.087125,-0.014105,-0.033465,-0.097658,-0.005426,0.024526,0.053337,-0.024528,...,0.004608,0.051395,-0.131844,0.010030,-0.074501,-0.025963,1.000000,-0.123420,0.041274,0.033676
65dda7855e43b9d7d3e4fee8,-0.044066,-0.017556,0.130384,0.128684,0.106229,0.012760,0.084639,-0.029640,-0.017982,-0.139382,...,-0.069710,0.018426,-0.037999,-0.081986,0.027393,-0.035056,-0.123420,1.000000,0.070013,0.024725


In [18]:
# NOTE: bellow here are all the code explanation that is done on 'get_recommendations' function
# filters the DataFrame searched_vehicles to get only the rows where the user_id matches the given user_id 8. It extracts all reviews made by the user.
user_id = 8
user_reviews = vehicles_with_reviews[vehicles_with_reviews['user_id'] == user_id]
user_reviews

,review_id,vehicle_id,user_id,rating,travel_id,departure_at,from,to
198,65dda7bea276ad551e8d5675,65dda77d5e43b9d7d3e4f977,8,2,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
217,65dda7bea276ad551e8d5d09,65dda77d5e43b9d7d3e4f977,8,3,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
766,65dda7c3a276ad551e8e3963,65dda77d5e43b9d7d3e4f977,8,4,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
1001,65dda7c5a276ad551e8e9ac3,65dda77d5e43b9d7d3e4f977,8,2,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
1033,65dda7c5a276ad551e8eaf08,65dda77d5e43b9d7d3e4f977,8,3,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
...,...,...,...,...,...,...,...,...
119070,65dda7bea276ad551e8d71d4,65dda7835e43b9d7d3e4fd25,8,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119127,65dda7c0a276ad551e8d899d,65dda7835e43b9d7d3e4fd25,8,5,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119320,65dda7c0a276ad551e8dd654,65dda7835e43b9d7d3e4fd25,8,2,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119392,65dda7c3a276ad551e8dedfc,65dda7835e43b9d7d3e4fd25,8,4,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"


In [19]:
# This line creates a Pandas Series called user_ratings. 
# It extracts the rating values from the user_reviews DataFrame and sets the vehicle_id values as the index of the Series. 
# This Series represents the ratings given by the user to different vehicles.
user_ratings = pd.Series(user_reviews.rating.values, index=user_reviews.vehicle_id.values)
# here user could have rated the same vehicle multiple times so, we will going to take an average of that
user_ratings = user_reviews.groupby('vehicle_id')['rating'].mean()
user_ratings.shape

(102,)

In [20]:
# geting round values to convert it into integer
# user_ratings = user_ratings.round().astype(int)
user_ratings

vehicle_id
65dda7755e43b9d7d3e4f3f9    3.625000
65dda7765e43b9d7d3e4f4bf    3.600000
65dda7775e43b9d7d3e4f4d3    3.166667
65dda7775e43b9d7d3e4f4e6    3.250000
65dda7775e43b9d7d3e4f4ea    3.400000
                              ...   
65dda7855e43b9d7d3e4fead    2.000000
65dda7855e43b9d7d3e4fecb    3.666667
65dda7855e43b9d7d3e4fee8    3.600000
65dda7865e43b9d7d3e4ff06    3.600000
65dda7865e43b9d7d3e4ff24    5.000000
Name: rating, Length: 102, dtype: float64

In [21]:
## Let's make recommendations
# This function taks vehicle_id & the rating that the auth user ex '8' have given on the vehicle_id on the past
def get_similar_vehicles(vehicle_id,user_rating):
    # let's get row of datafrom from 'vehicles_similarity_matrix_df' using the index 'vehicle_id' 
    # which means get row from the vehicle which user have already rated
    similar_score = vehicles_similarity_matrix_df[vehicle_id]
    # Now scale it by the rating that the user have given for that particular vehicle
#     similar_score = similar_score*user_rating
#     now arranging it in decending order
    similar_score = similar_score*(user_rating-2.5) # explanation is bellow next cell from this cell
    similar_score = similar_score.sort_values(ascending=False)
    # This method will return a similarity score for all the vehicles that is similar to the given vehicle 'vehicle_id'
    return similar_score



In [22]:
# Testing for the first user rated vehicle
first_vehicle_id = user_ratings.index[0]
first_vehicle_rating = user_ratings.iloc[0]


get_similar_vehicles(first_vehicle_id, first_vehicle_rating)
# Let's say that user give rating 1 to vehicle 'first_vehicle_id' 
# Explanation: https://youtu.be/3ecNC-So0r4?t=1225
# If the user have rated the vehicles bad then we want all other similar vehicles to go down in the list
# and other vehicle that is not similar to this vehicle to come at the top of the list
# we can achieve this in above function like this:
# similar_score = similar_score*(user_rating-2.5)
get_similar_vehicles(first_vehicle_id, 1)

vehicle_id
65dda7775e43b9d7d3e4f54d    0.286131
65dda7785e43b9d7d3e4f5cd    0.275026
65dda7775e43b9d7d3e4f53f    0.254408
65dda77a5e43b9d7d3e4f712    0.243421
65dda77c5e43b9d7d3e4f86f    0.202321
                              ...   
65dda7805e43b9d7d3e4fb98   -0.227551
65dda77e5e43b9d7d3e4f9c1   -0.251646
65dda7765e43b9d7d3e4f4bf   -0.297529
65dda77c5e43b9d7d3e4f87f   -0.379955
65dda7755e43b9d7d3e4f3f9   -1.500000
Name: 65dda7755e43b9d7d3e4f3f9, Length: 103, dtype: float64

In [23]:
# converting 'user_ratings' into list of touples
user_ratings_list = list(user_ratings.items())
user_ratings_list

[('65dda7755e43b9d7d3e4f3f9', 3.625),
 ('65dda7765e43b9d7d3e4f4bf', 3.6),
 ('65dda7775e43b9d7d3e4f4d3', 3.1666666666666665),
 ('65dda7775e43b9d7d3e4f4e6', 3.25),
 ('65dda7775e43b9d7d3e4f4ea', 3.4),
 ('65dda7775e43b9d7d3e4f4ef', 3.3333333333333335),
 ('65dda7775e43b9d7d3e4f4fb', 4.0),
 ('65dda7775e43b9d7d3e4f513', 3.090909090909091),
 ('65dda7775e43b9d7d3e4f529', 3.125),
 ('65dda7775e43b9d7d3e4f534', 3.8),
 ('65dda7775e43b9d7d3e4f53f', 3.8333333333333335),
 ('65dda7775e43b9d7d3e4f54d', 3.0833333333333335),
 ('65dda7775e43b9d7d3e4f557', 3.6666666666666665),
 ('65dda7775e43b9d7d3e4f569', 3.857142857142857),
 ('65dda7775e43b9d7d3e4f570', 3.0),
 ('65dda7785e43b9d7d3e4f583', 3.0),
 ('65dda7785e43b9d7d3e4f58b', 3.25),
 ('65dda7785e43b9d7d3e4f5a7', 3.375),
 ('65dda7785e43b9d7d3e4f5c2', 3.6),
 ('65dda7785e43b9d7d3e4f5cd', 3.4444444444444446),
 ('65dda7785e43b9d7d3e4f5df', 2.5714285714285716),
 ('65dda7785e43b9d7d3e4f5e1', 4.166666666666667),
 ('65dda7785e43b9d7d3e4f5ff', 2.0),
 ('65dda7785e43b9

In [24]:
# Now we now all the vehicle that auth user have rated is stored on 'user_ratings'

similar_vehicles_df = pd.DataFrame()

# Initialize a list to store the DataFrames for each vehicle
dfs = []

# Now we will going to get similarities from every vehicle that user have rated to every other vehicles
for vehicle_id, rating in user_ratings_list:
    # Get similarity DataFrame for the current vehicle and rating
    similarity_df = get_similar_vehicles(vehicle_id, rating)
    # Set the column name to the vehicle_id
    similarity_df.columns = [vehicle_id]
    # Append the DataFrame to the list
    dfs.append(similarity_df)
    
# Concatenate all DataFrames in the list along axis 1
similar_vehicles_df = pd.concat(dfs, axis=1)

# Set the index to be the default integer index
similar_vehicles_df.reset_index(drop=True, inplace=True)
# Each row in this dataframe for each 'vehicle_id' that the user has rated
# And the value in each column for vehicle id is the similarity score for each row index 'vehicle_id'
# It means that every value in this dataframe are the similarity value for each vehicle that user have rated on
similar_vehicles_df

,65dda7755e43b9d7d3e4f3f9,65dda7765e43b9d7d3e4f4bf,65dda7775e43b9d7d3e4f4d3,65dda7775e43b9d7d3e4f4e6,65dda7775e43b9d7d3e4f4ea,65dda7775e43b9d7d3e4f4ef,65dda7775e43b9d7d3e4f4fb,65dda7775e43b9d7d3e4f513,65dda7775e43b9d7d3e4f529,65dda7775e43b9d7d3e4f534,...,65dda7845e43b9d7d3e4fe17,65dda7845e43b9d7d3e4fe34,65dda7855e43b9d7d3e4fe52,65dda7855e43b9d7d3e4fe6f,65dda7855e43b9d7d3e4fe8c,65dda7855e43b9d7d3e4fead,65dda7855e43b9d7d3e4fecb,65dda7855e43b9d7d3e4fee8,65dda7865e43b9d7d3e4ff06,65dda7865e43b9d7d3e4ff24
0,1.125000,0.218188,0.071982,0.037591,-0.116687,0.021982,-0.008445,0.011789,-0.039170,-0.020514,...,-0.0,0.007734,-0.067388,0.025940,0.228149,-0.073572,-0.041261,-0.048473,0.138577,0.063021
1,0.284966,0.055004,0.006568,0.104794,-0.048654,-0.077748,0.120483,-0.054963,0.002620,0.019982,...,0.0,0.009011,-0.160526,0.069529,0.031838,0.031000,0.078036,-0.054247,0.071423,-0.036397
2,0.223147,1.100000,-0.015988,-0.023035,-0.038904,0.007628,0.035908,0.006067,0.006991,0.129936,...,-0.0,0.036434,0.077645,0.068576,0.011528,-0.014741,0.034305,-0.019312,0.032150,-0.040791
3,0.188735,0.021407,-0.005694,-0.058181,0.077493,-0.007479,-0.076087,0.052181,-0.025835,-0.097460,...,-0.0,0.004146,-0.060834,-0.162029,-0.017885,-0.102145,0.026933,-0.028583,-0.057523,0.129488
4,0.170663,-0.171014,-0.006160,-0.012769,0.072562,-0.113372,-0.013813,-0.035699,0.013906,-0.062731,...,-0.0,-0.049972,-0.081203,-0.159653,0.064822,-0.061492,-0.028548,0.003250,0.162655,0.326340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,-0.151741,-0.173085,-0.039935,0.064788,0.061804,0.029908,-0.008297,-0.016793,-0.065116,-0.135664,...,-0.0,0.076944,0.043314,0.057912,-0.074540,0.006973,-0.000505,0.054050,0.017887,0.205992
99,-0.182566,-0.048828,-0.052965,-0.002973,-0.059608,-0.050359,0.013814,-0.037907,-0.005116,0.055964,...,0.0,-0.040263,0.007188,0.093861,0.049957,-0.034674,0.065360,-0.061153,0.045839,-0.017223
100,-0.190806,-0.096116,-0.001108,-0.000771,-0.147479,0.053627,-0.033592,-0.044664,0.048811,-0.073863,...,-0.0,0.028454,-0.057930,0.040419,-0.077382,-0.009010,0.055534,0.056669,0.016202,0.012257
101,-0.206270,-0.153101,-0.076187,0.036238,-0.051041,-0.134165,-0.066617,0.058577,-0.028768,0.093194,...,0.0,0.004148,-0.024131,-0.099462,-0.021410,-0.034685,0.040166,-0.031915,0.096897,0.059055


In [25]:
# Now let's sum all the vehicle id values row wise and sort them in decending order
similar_vehicles = similar_vehicles_df.sum().sort_values(ascending=False)
similar_vehicles

65dda7835e43b9d7d3e4fd28    4.404907
65dda7865e43b9d7d3e4ff24    3.884208
65dda7855e43b9d7d3e4fe8c    3.490505
65dda77e5e43b9d7d3e4f9e4    3.086571
65dda7755e43b9d7d3e4f3f9    3.080462
                              ...   
65dda7805e43b9d7d3e4fb02   -0.220296
65dda7795e43b9d7d3e4f684   -0.347906
65dda7785e43b9d7d3e4f5ff   -0.723686
65dda7855e43b9d7d3e4fead   -0.863637
65dda7825e43b9d7d3e4fc48   -0.908247
Length: 102, dtype: float64

In [26]:
recommended_vehicles = similar_vehicles.index.tolist()

In [27]:
recommended_vehicles[:25]

['65dda7835e43b9d7d3e4fd28',
 '65dda7865e43b9d7d3e4ff24',
 '65dda7855e43b9d7d3e4fe8c',
 '65dda77e5e43b9d7d3e4f9e4',
 '65dda7755e43b9d7d3e4f3f9',
 '65dda77e5e43b9d7d3e4fa06',
 '65dda77d5e43b9d7d3e4f952',
 '65dda7835e43b9d7d3e4fd45',
 '65dda77a5e43b9d7d3e4f785',
 '65dda7845e43b9d7d3e4fdba',
 '65dda7815e43b9d7d3e4fbbd',
 '65dda7845e43b9d7d3e4fdfb',
 '65dda7855e43b9d7d3e4fe6f',
 '65dda77c5e43b9d7d3e4f84e',
 '65dda7785e43b9d7d3e4f5e1',
 '65dda77d5e43b9d7d3e4f92e',
 '65dda77d5e43b9d7d3e4f8e8',
 '65dda7825e43b9d7d3e4fcc7',
 '65dda7865e43b9d7d3e4ff06',
 '65dda7805e43b9d7d3e4fb73',
 '65dda7785e43b9d7d3e4f61f',
 '65dda77f5e43b9d7d3e4fa94',
 '65dda77a5e43b9d7d3e4f708',
 '65dda7825e43b9d7d3e4fc89',
 '65dda77b5e43b9d7d3e4f7cf']

In [28]:
# NOTE: Issues that we could face on Collaborative Filtering
# 1. Handling Unknown Users / Vehicles (Cold Start Problem) where we don't know any context about it for rignt now
# 2. Scalalability: Takes havy performance to perform this model
# 3. Dynamic Update: When ever we will add new vehicle or User or Rating Data Then we again have to re build this model
# For that we can create a Hybrid recommendation system to handle this challanges
# Also we can use Papularity based filtering

In [29]:
# Now we will export all the trained dataset into pkl files
import pickle
pickle.dump(vehicles_with_reviews, open("vehicles_with_reviews.pkl", 'wb'))

# Popularity Based Recommended System

In [ ]:
# Check whether given user id exist or not or is it None value if not then use Popularity Based recommendation
anonymouse_user_id = None
anonymouse_user_id = 430
anonymous_user_reviews = searched_vehicles[searched_vehicles['user_id'] == anonymouse_user_id]
anonymous_user_reviews.count()["rating"]

In [41]:
# Find rating coung of every vehicles
vehicles_with_ratings_df = vehicles_with_reviews.groupby("vehicle_id").count()["rating"].reset_index()
vehicles_with_ratings_df.rename(columns={"rating":"no_of_rating"}, inplace=True)
vehicles_with_ratings_df

,vehicle_id,no_of_rating
0,65dda7755e43b9d7d3e4f3f9,1221
1,65dda7765e43b9d7d3e4f4bf,1176
2,65dda7775e43b9d7d3e4f4d3,1193
3,65dda7775e43b9d7d3e4f4e6,1146
4,65dda7775e43b9d7d3e4f4ea,1122
...,...,...
98,65dda7855e43b9d7d3e4fead,1194
99,65dda7855e43b9d7d3e4fecb,1157
100,65dda7855e43b9d7d3e4fee8,1183
101,65dda7865e43b9d7d3e4ff06,1238


In [42]:
# Find Average rating of every vehicles
average_rating_df = vehicles_with_reviews.groupby("vehicle_id")["rating"].mean().reset_index()
average_rating_df.rename(columns={"rating":"avg_rating"}, inplace=True)
average_rating_df

,vehicle_id,avg_rating
0,65dda7755e43b9d7d3e4f3f9,3.486486
1,65dda7765e43b9d7d3e4f4bf,3.460034
2,65dda7775e43b9d7d3e4f4d3,3.516345
3,65dda7775e43b9d7d3e4f4e6,3.511344
4,65dda7775e43b9d7d3e4f4ea,3.482175
...,...,...
98,65dda7855e43b9d7d3e4fead,3.489950
99,65dda7855e43b9d7d3e4fecb,3.532411
100,65dda7855e43b9d7d3e4fee8,3.441251
101,65dda7865e43b9d7d3e4ff06,3.508078


In [43]:
# Merge average rating & rating count
popular_df = vehicles_with_ratings_df.merge(average_rating_df,on='vehicle_id')
popular_df

,vehicle_id,no_of_rating,avg_rating
0,65dda7755e43b9d7d3e4f3f9,1221,3.486486
1,65dda7765e43b9d7d3e4f4bf,1176,3.460034
2,65dda7775e43b9d7d3e4f4d3,1193,3.516345
3,65dda7775e43b9d7d3e4f4e6,1146,3.511344
4,65dda7775e43b9d7d3e4f4ea,1122,3.482175
...,...,...,...
98,65dda7855e43b9d7d3e4fead,1194,3.489950
99,65dda7855e43b9d7d3e4fecb,1157,3.532411
100,65dda7855e43b9d7d3e4fee8,1183,3.441251
101,65dda7865e43b9d7d3e4ff06,1238,3.508078


In [49]:
# First sort with 'no_of_rating' and only get 30 or them
popular_df = popular_df.sort_values('no_of_rating',ascending=False).head(30)
popular_df.shape

(30, 3)

In [52]:
# Now sort with 'avg_rating'
popular_df = popular_df.sort_values('avg_rating',ascending=False)
popular_df

,vehicle_id,no_of_rating,avg_rating
28,65dda7795e43b9d7d3e4f661,1236,3.536408
62,65dda77e5e43b9d7d3e4fa29,1204,3.524917
76,65dda7815e43b9d7d3e4fc26,1188,3.521044
84,65dda7835e43b9d7d3e4fd25,1198,3.520868
44,65dda77b5e43b9d7d3e4f817,1237,3.520614
83,65dda7835e43b9d7d3e4fd04,1195,3.518828
2,65dda7775e43b9d7d3e4f4d3,1193,3.516345
25,65dda7795e43b9d7d3e4f63b,1191,3.512175
47,65dda77c5e43b9d7d3e4f85c,1229,3.511798
101,65dda7865e43b9d7d3e4ff06,1238,3.508078


In [57]:
popular_vehicles = popular_df["vehicle_id"]
popular_vehicles = popular_vehicles.values.tolist()

In [58]:
popular_vehicles

['65dda7795e43b9d7d3e4f661',
 '65dda77e5e43b9d7d3e4fa29',
 '65dda7815e43b9d7d3e4fc26',
 '65dda7835e43b9d7d3e4fd25',
 '65dda77b5e43b9d7d3e4f817',
 '65dda7835e43b9d7d3e4fd04',
 '65dda7775e43b9d7d3e4f4d3',
 '65dda7795e43b9d7d3e4f63b',
 '65dda77c5e43b9d7d3e4f85c',
 '65dda7865e43b9d7d3e4ff06',
 '65dda77c5e43b9d7d3e4f87f',
 '65dda7805e43b9d7d3e4fb98',
 '65dda7825e43b9d7d3e4fcc7',
 '65dda77a5e43b9d7d3e4f708',
 '65dda77f5e43b9d7d3e4fade',
 '65dda7785e43b9d7d3e4f583',
 '65dda7815e43b9d7d3e4fbbd',
 '65dda77d5e43b9d7d3e4f94f',
 '65dda7855e43b9d7d3e4fe8c',
 '65dda7855e43b9d7d3e4fead',
 '65dda7755e43b9d7d3e4f3f9',
 '65dda7775e43b9d7d3e4f534',
 '65dda7835e43b9d7d3e4fd7f',
 '65dda7785e43b9d7d3e4f5c2',
 '65dda7775e43b9d7d3e4f557',
 '65dda7795e43b9d7d3e4f6a7',
 '65dda77f5e43b9d7d3e4fab9',
 '65dda77a5e43b9d7d3e4f739',
 '65dda7785e43b9d7d3e4f58b',
 '65dda7825e43b9d7d3e4fca6']